In [1]:
import torch
from simpletransformers.t5 import T5Model, T5Args

In [2]:
cuda =  torch.cuda.is_available()
cuda

True

In [3]:
# parameters for prediction, set multiprocessed decoding to False to avoid CUDA out of memory error
pred_params = {
        'max_seq_length': 512,
        'use_multiprocessed_decoding': False
        }

#load final model from /outputs
model = T5Model('t5', 'outputs/best_model', args=pred_params, use_cuda=cuda) 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import pandas as pd
test =  pd.read_csv('test.csv')

In [7]:
# rename columns to match the model's input
test = test.rename(columns={"summary":"target_text", "text":"input_text"})
test['prefix'] = ''
test

,input_text,target_text,prefix
0,"washington, mar (pti) a -year-old woman i...","a woman, who was arrested twice last week for ...",
1,the tea stall in guajarat's vadnagar where pri...,a tea stall in gujarat's vadnagar railway stat...,
2,"continuing a yearly tradition, outgoing us pre...",us president barack obama has declared january...,
3,the supreme court on thursday rejected a petit...,the supreme court on thursday rejected a petit...,
4,"captain amarinder singh, the congress's chief ...",congress leader captain amarinder singh will t...,
...,...,...,...
875,"new delhi, dec (pti) dismissing the petition...",while dismissing a petition against demonetisa...,
876,"there's strange, there's unfathomable, then th...",a dutch game show was slammed by people for an...,
877,taimur ali khan is undoubtedly one of the cute...,pictures of saif ali khan and kareena kapoor k...,
878,a preliminary enquiry has been ordered by the...,the economic offences wing of the delhi police...,


In [8]:
# make predictions from our model

pred = model.predict(list(test['input_text']))

Generating outputs:   0%|          | 0/110 [00:00<?, ?it/s]

c:\Users\Perplex420\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:4072: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
c:\Users\Perplex420\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers

In [9]:
import random

In [10]:


random.sample(pred, 5)

['filmmaker neeraj ghaywan has said that the issue of nepotism has been overblown and has turned pointless now. he added that the issue (of nepotism) has turned pointless now. he added that the issue (of nepotism) has turned pointless now. neeraj added that i agree that we need acknowledge that it exists and there is a certain set of privileges that these actors enjoy.',
 'the all india council for technical education (aicte) has asked all the institutions affiliated to it to "urgencely put in place an online mechanism" for registering and disposing of grievances. according to a recent aicte directive, the hrd ministry has stressed the need to have a mechanism for online registration as well as disposal of the grievances not only of the students but also the faculty members.',
 "just companies have been the source of more than % of the world's greenhouse gas emissions since, according to a new carbon majors report. the carbon majors report aims to improve transparency among fossil fuel

In [12]:
# using sumeval to calculate ROUGE scores , faster way to calculate ROUGE scores

from sumeval.metrics.rouge import RougeCalculator
rouge = RougeCalculator(stopwords=True, lang="en")

In [13]:
import numpy as np

In [14]:
# function to calculate ROUGE scores

def rouge_calc(preds, targets):
    rouge_1 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=1) for i in range(len(preds))]
    rouge_2 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=2) for i in range(len(preds))]
    rouge_l = [rouge.rouge_l(summary=preds[i],references=targets[i]) for i in range(len(preds))]

    return {"Rouge_1": np.array(rouge_1).mean(),
            "Rouge_2": np.array(rouge_2).mean(),
            "Rouge_L": np.array(rouge_l).mean()}

In [15]:
#final ROUGE scores for the model

rouge_calc(pred, list(test['target_text']))

{'Rouge_1': 0.4200905043366553,
 'Rouge_2': 0.2237638177289328,
 'Rouge_L': 0.3491328918546062}